Kaggle Dataset pet finder


#Instructions

Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import random, convert_to_tensor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, BatchNormalization, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import get_file
from tensorflow.keras.callbacks import EarlyStopping
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import sklearn as sk
import tensorflow as tf
from datetime import datetime
from packaging import version
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorboard
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")



In [ ]:
%load_ext tensorboard
tf.summary.trace_on(
    graph=True, profiler=False
)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
np.random.seed(0)
random.set_seed(0)

In [ ]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

get_file('petfinder_mini.zip', dataset_url,
                        extract=True, cache_dir='.')
dataset = pd.read_csv(csv_file)

In [ ]:
type(dataset)

pandas.core.frame.DataFrame

In [ ]:
dataset.head

<bound method NDFrame.head of       Type  Age  ... PhotoAmt AdoptionSpeed
0      Cat    3  ...        1             2
1      Cat    1  ...        2             0
2      Dog    1  ...        7             3
3      Dog    4  ...        8             2
4      Dog    1  ...        3             2
...    ...  ...  ...      ...           ...
11532  Dog   24  ...        0             4
11533  Cat    1  ...        1             3
11534  Dog    6  ...        1             0
11535  Cat    9  ...        3             4
11536  Dog    1  ...        1             3

[11537 rows x 15 columns]>

In [ ]:
dataset.columns

Index(['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'Description',
       'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')

In [ ]:
dataset['Target']=np.where(dataset['AdoptionSpeed']==4 ,0,1)

In [ ]:
dataset.columns

Index(['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'Description',
       'PhotoAmt', 'AdoptionSpeed', 'Target'],
      dtype='object')

In [ ]:
g = dataset.columns.to_series().groupby(dataset.dtypes).groups

In [ ]:
g

{int64: ['Age', 'Fee', 'PhotoAmt', 'AdoptionSpeed', 'Target'], object: ['Type', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Description']}

In [ ]:
g1= {k.name: v for k, v in g.items()}
g1

{'int64': Index(['Age', 'Fee', 'PhotoAmt', 'AdoptionSpeed', 'Target'], dtype='object'),
 'object': Index(['Type', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
        'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Description'],
       dtype='object')}

In [ ]:
g1['int64']

Index(['Age', 'Fee', 'PhotoAmt', 'AdoptionSpeed', 'Target'], dtype='object')

In [ ]:
g1['object']

Index(['Type', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Description'],
      dtype='object')

We need to convert all object which is categorical to int , for tensorflow to understand , we will use cat which is series function 

In [ ]:
#dataset['Type'] = dataset['Type'].astype('category')
#dataset['Type'] = dataset['Type'].cat.codes
#this will convert from Dog , Cat to 1,2

In [ ]:
# all the columns with type -object will be listed
cat_features = list(g1["object"])
cat_features

['Type',
 'Breed1',
 'Gender',
 'Color1',
 'Color2',
 'MaturitySize',
 'FurLength',
 'Vaccinated',
 'Sterilized',
 'Health',
 'Description']

In [ ]:
g2 = dataset.columns.to_series().groupby(dataset.dtypes).groups
g2

{int64: ['Age', 'Fee', 'PhotoAmt', 'AdoptionSpeed', 'Target'], object: ['Type', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Description']}

In [ ]:
cat_features = list(g1["object"])
cat_features

['Type',
 'Breed1',
 'Gender',
 'Color1',
 'Color2',
 'MaturitySize',
 'FurLength',
 'Vaccinated',
 'Sterilized',
 'Health',
 'Description']

In [ ]:
for column in cat_features:

    #first step is to convert column from object to categorical 
    dataset[column] = dataset[column].astype('category')
    #second step is use the pandas series function to cat to convert it into codes
    dataset[column] = dataset[column].cat.codes

#complete list of columns converted to float64
dataset = dataset.astype(np.float64)

In [ ]:
g3 = dataset.columns.to_series().groupby(dataset.dtypes).groups
g3

{float64: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'Description', 'PhotoAmt', 'AdoptionSpeed', 'Target']}

In [ ]:
train_data ,test_data=sk.model_selection.train_test_split(dataset,test_size=.2,random_state=0)

In [ ]:
X_train=train_data.iloc[:,:-1]
y_train=train_data.iloc[:,-1]

In [ ]:
y_train

1382     0.0
8302     1.0
5961     1.0
6225     1.0
4384     1.0
        ... 
4859     1.0
3264     1.0
9845     0.0
10799    1.0
2732     1.0
Name: Target, Length: 9229, dtype: float64

In [ ]:
X_test=test_data.iloc[:,:-1]
y_test=test_data.iloc[:,-1]

In [ ]:
X_train_tensor=tf.constant(X_train)
y_train_tensor=tf.constant(y_train)
X_test_tensor=tf.constant(X_test)
y_test_tensor=tf.constant(y_test)

In [ ]:
type(X_train_tensor)

tensorflow.python.framework.ops.EagerTensor

In [ ]:
type(X_train_tensor)

tensorflow.python.framework.ops.EagerTensor

In [ ]:
X_train_tensor

<tf.Tensor: shape=(9229, 15), dtype=float64, numpy=
array([[1.000e+00, 2.400e+01, 9.900e+01, ..., 1.154e+03, 7.000e+00,
        4.000e+00],
       [1.000e+00, 2.000e+00, 8.900e+01, ..., 6.273e+03, 2.000e+00,
        3.000e+00],
       [1.000e+00, 3.000e+00, 9.900e+01, ..., 2.375e+03, 1.000e+00,
        3.000e+00],
       ...,
       [1.000e+00, 1.200e+01, 1.220e+02, ..., 3.714e+03, 1.000e+01,
        4.000e+00],
       [1.000e+00, 1.400e+01, 1.320e+02, ..., 7.157e+03, 6.000e+00,
        1.000e+00],
       [1.000e+00, 1.000e+00, 9.900e+01, ..., 8.156e+03, 7.000e+00,
        3.000e+00]])>

In [ ]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(units=6,activation='relu',input_shape=(1,)))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(x=X_train_tensor,y=y_train_tensor, batch_size=32 ,epochs = 20, validation_data=(X_test_tensor,y_test_tensor))

Epoch 1/20
289/289 [==============================] - 1s 2ms/step - loss: 0.5831 - accuracy: 0.6818 - val_loss: 0.4466 - val_accuracy: 0.7166
Epoch 2/20
289/289 [==============================] - 1s 2ms/step - loss: 0.3854 - accuracy: 0.7546 - val_loss: 0.2905 - val_accuracy: 0.8657
Epoch 3/20
289/289 [==============================] - 1s 2ms/step - loss: 0.2629 - accuracy: 0.9002 - val_loss: 0.2053 - val_accuracy: 0.9705
Epoch 4/20
289/289 [==============================] - 1s 2ms/step - loss: 0.1876 - accuracy: 0.9619 - val_loss: 0.1018 - val_accuracy: 0.9913
Epoch 5/20
289/289 [==============================] - 1s 2ms/step - loss: 0.0969 - accuracy: 0.9844 - val_loss: 0.0402 - val_accuracy: 0.9987
Epoch 6/20
289/289 [==============================] - 1s 2ms/step - loss: 0.0549 - accuracy: 0.9900 - val_loss: 0.0195 - val_accuracy: 1.0000
Epoch 7/20
289/289 [==============================] - 1s 2ms/step - loss: 0.0414 - accuracy: 0.9912 - val_loss: 0.0106 - val_accuracy: 1.0000
Epoch 

In [ ]:
y_test=model.predict(X_test_tensor)

In [ ]:
y_test=np.array(y_test)

In [ ]:
y_test_tensor=np.array(y_test_tensor)

In [ ]:
y_test=np.where(y_test> .5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_test_tensor)
print(cm)
accuracy_score(y_test, y_test_tensor)


[[ 642    0]
 [   0 1666]]


1.0

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 15)                60        
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 96        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 163
Trainable params: 133
Non-trainable params: 30
_________________________________________________________________
